SVM + Word2Vec Classifier
Classify Attack Vector by learning from descriptions.
Classes:  network, adjacent network, local, physical.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import spacy
import seaborn as sns

from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

nlp = spacy.load("en_core_web_lg")

In [32]:
data_train = pd.read_csv('cve_train.csv')
data_test = pd.read_csv('cve_test.csv')

description_train = data_train['description']
description_test = data_test['description']
y_train = data_train['integrity']
y_test = data_test['integrity']

In [4]:
len(description_test.index)

15404

In [3]:
vectors = description_train.apply(lambda row: sum([token.vector for token in nlp(row)]))

In [4]:
%store vectors 

Stored 'vectors' (Series)


In [ ]:
%store -r vectors

In [33]:
description_train = list(vectors)
clf = make_pipeline(StandardScaler(), svm.SVC(gamma='auto'))
clf.fit(description_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [6]:
list_desc = []
for i in range(len(description_test)):
    list_desc.append(description_test.iloc[i])

In [7]:
list_desc_vec = []
for desc in list_desc:
    list_desc_vec.append(sum([token.vector for token in nlp(desc)]))

In [8]:
%store list_desc_vec

Stored 'list_desc_vec' (list)


In [ ]:
%store -r list_desc_vec

In [34]:
y_pred = []
for desc_vec in list_desc_vec:
    y_pred.append(clf.predict([desc_vec]))

In [35]:
y_true = []
for i in range(len(y_test)):
    y_true.append(y_test.iloc[i])

In [36]:
target_names = ['HIGH', 'LOW', 'NONE']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

        HIGH       0.83      0.92      0.87      7895
         LOW       0.95      0.77      0.85      2693
        NONE       0.85      0.79      0.82      4816

    accuracy                           0.85     15404
   macro avg       0.88      0.82      0.85     15404
weighted avg       0.86      0.85      0.85     15404



In [10]:
import pickle
pickle.dump(clf, open('svm_model', 'wb'))